## Dependencies 

In [8]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import time
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import os
import gmaps

# Import gkey
from config import gkey

## Create a DF Containing Data from Weather CSV

In [6]:
#Find where CSV file is and read it into this notebook
csv_file = os.path.join('..', 'Output_Data', 'weather.csv')

#Turn it into a new DF
weather_df = pd.read_csv(csv_file)
weather_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Qaanaaq,100,GL,82,77.4840,-69.3632,-0.81,2.28
1,Morąg,71,PL,90,53.9171,19.9260,30.99,3.00
2,Dikson,92,RU,100,73.5069,80.5464,-7.33,6.62
3,Beringovskiy,10,RU,83,63.0500,179.3167,0.57,8.03
4,Castro,49,BR,94,-24.7911,-50.0119,59.20,4.65
...,...,...,...,...,...,...,...,...
474,Coquimbo,0,CL,77,-29.9533,-71.3436,62.60,11.50
475,Ténenkou,52,ML,9,14.4572,-4.9169,88.02,10.98
476,Gamboula,58,CF,71,4.1178,15.1393,73.15,3.22
477,Meulaboh,99,ID,85,4.1363,96.1285,76.80,2.62


## Start Creating Heatmaps

### • Humidity From Every City Heatmap

In [10]:
# Configure gkey
gmaps.configure(api_key=gkey)

In [12]:
#Set Latitude & Longitude to be the locations in the heatmap and Humidity as the weight
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [26]:
#Create Heat Layer and add it to Google Map
fig = gmaps.figure(center=(30,20), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 1.5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### • Ideal Weather Condition Heatmap 

#### Ideal Conditions:
• Temperature between 75° and 90° <br/>
• Wind Speed < 15 MPH <br/>
• Cloudiness < 15% <br/>

In [41]:
# Create DF with Relevant Data Filtered
ideal_df = weather_df[(weather_df['Max Temp'] >= 75) & (weather_df['Max Temp'] <= 90)]
ideal_df = ideal_df[weather_df['Wind Speed'] <= 15]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 15]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
ideal_df.head()

C:\Users\alexz\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
7,Avera,1,US,24,33.1940,-82.5271,80.60,5.75
18,Rikitea,9,PF,70,-23.1203,-134.9692,77.29,8.88
37,Karratha,5,AU,53,-20.7377,116.8463,80.42,7.40
85,Saint-Philippe,0,RE,69,-21.3585,55.7679,77.00,4.61
93,Saint-Joseph,0,RE,69,-21.3667,55.6167,77.00,4.61


In [42]:
#### Ceate new DF to add Hotel Column into
hotel_df = ideal_df
hotel_df.head()

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
7,Avera,1,US,24,33.1940,-82.5271,80.60,5.75
18,Rikitea,9,PF,70,-23.1203,-134.9692,77.29,8.88
37,Karratha,5,AU,53,-20.7377,116.8463,80.42,7.40
85,Saint-Philippe,0,RE,69,-21.3585,55.7679,77.00,4.61
93,Saint-Joseph,0,RE,69,-21.3667,55.6167,77.00,4.61


In [44]:
#Add in  new column where Hotel Name will go
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Avera,1,US,24,33.1940,-82.5271,80.60,5.75,
18,Rikitea,9,PF,70,-23.1203,-134.9692,77.29,8.88,
37,Karratha,5,AU,53,-20.7377,116.8463,80.42,7.40,
85,Saint-Philippe,0,RE,69,-21.3585,55.7679,77.00,4.61,
93,Saint-Joseph,0,RE,69,-21.3667,55.6167,77.00,4.61,


In [45]:
#Create the base URL for the API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set parameters for search
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": gkey
}

In [46]:
# Make API Calls in Loop for each set of coordinates and find a hotel within 5000 meters
for index, row in hotel_df.iterrows():
    # Set latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Set city coordinates to equal the Lat & Lng
    coordinates = f"{lat}, {lng}"
    
    #Add these to the API Search
    params["location"] = coordinates
    
    # Make the API Call
    response = requests.get(base_url, params=params).json()
    # Load results   
    results = response['results']
 
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f'The nearest hotel to {row["City"]} is {results[0]["name"]}')
        
    except:
        print("Could not find a hotel within range")
    
    time.sleep(1)

Could not find a hotel within range
Could not find a hotel within range
The nearest hotel to Karratha is Aspen Karratha Village
The nearest hotel to Saint-Philippe is Les Embruns Du Baril
The nearest hotel to Saint-Joseph is "Plantation Bed and Breakfast
Could not find a hotel within range
The nearest hotel to Saint-Pierre is Le Battant Des Lames
Could not find a hotel within range
Could not find a hotel within range
The nearest hotel to Tomatlán is Hotel Misión San Miguel
The nearest hotel to Katsina is Liyafa Palace Hotel
Could not find a hotel within range
The nearest hotel to Sur is Sur Plaza Hotel
Could not find a hotel within range
Could not find a hotel within range
The nearest hotel to Buin is Paihuen
The nearest hotel to Eilat is Kempinski Hotel Aqaba Red Sea
The nearest hotel to Diffa is Complexe Hôtelier Du 18 Décembre
The nearest hotel to Bhindār is Hotel Rajmahal Bhindar
The nearest hotel to Laguna is Holiday Inn Express & Suites Elk Grove West I-5
The nearest hotel to Rid

In [47]:
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Avera,1,US,24,33.1940,-82.5271,80.60,5.75,
18,Rikitea,9,PF,70,-23.1203,-134.9692,77.29,8.88,
37,Karratha,5,AU,53,-20.7377,116.8463,80.42,7.40,Aspen Karratha Village
85,Saint-Philippe,0,RE,69,-21.3585,55.7679,77.00,4.61,Les Embruns Du Baril
93,Saint-Joseph,0,RE,69,-21.3667,55.6167,77.00,4.61,"""Plantation Bed and Breakfast"
95,Goundam,0,ML,5,16.4145,-3.6708,82.90,9.71,
103,Saint-Pierre,0,RE,69,-21.3393,55.4781,77.00,4.61,Le Battant Des Lames
123,Araouane,0,ML,7,18.9048,-3.5265,81.82,7.70,
125,Sankheda,0,IN,36,22.1667,73.5833,80.64,2.39,
135,Tomatlán,0,MX,37,19.9333,-105.2500,84.43,8.77,Hotel Misión San Miguel
